In [ ]:
from pyspark.sql import types
from pyspark.sql import functions as F
from pyspark.sql import window

from delta.tables import *

In [ ]:
def check_table_exist(db_tbl_name):
    table_exist = False
    try:
        spark.read.table(db_tbl_name) # Check if spark can read the table
        table_exist = True        
    except:
        pass
    return table_exist

In [ ]:
%sql set spark.databricks.delta.properties.defaults.enableChangeDataFeed = true;

key,value
spark.databricks.delta.properties.defaults.enableChangeDataFeed,true


In [ ]:
table = "players.csv"
bronze_table = f'bronze.{table.split(".")[0]}table'
silver_table = f'silver.{table.split(".")[0]}table'
checkpoint_path = f"s3://csgo-datalake/silver/{table}_checkpoint"
table_name = f"{table.split('.')[0]}Table"


In [ ]:
id_field = ['player_id','team']
strongly_date =  'date'

In [ ]:
query =  """SELECT 
Op,
date,
player_name,
team,
opponent,
country,
player_id,
match_id,
event_id,
event_name,
best_of,
map_1,
map_2,
map_3,
kills,
assists,
deaths,
kills + assists - deaths as killad,
hs,
flash_assists,
kast,
kddiff,
adr,
fkdiff,
rating,
m1_kills,
m1_assists,
m1_deaths,
m1_hs,
m1_flash_assists,
m1_kast,
m1_kddiff,
m1_adr,
m1_fkdiff,
m1_rating,
m2_kills,
m2_assists,
m2_deaths,
m2_hs,
m2_flash_assists,
m2_kast,
m2_kddiff,
m2_adr,
m2_fkdiff,
m2_rating,
m3_kills,
m3_assists,
m3_deaths,
m3_hs,
m3_flash_assists,
m3_kast,
m3_kddiff,
m3_adr,
m3_fkdiff,
m3_rating,
kills_ct,
deaths_ct,
kddiff_ct,
adr_ct,
kast_ct,
rating_ct,
kills_t,
deaths_t,
kddiff_t,
adr_t,
kast_t,
rating_t,
m1_kills_ct,
m1_deaths_ct,
m1_kddiff_ct,
m1_adr_ct,
m1_kast_ct,
m1_rating_ct,
m1_kills_t,
m1_deaths_t,
m1_kddiff_t,
m1_adr_t,
m1_kast_t,
m1_rating_t,
m2_kills_ct,
m2_deaths_ct,
m2_kddiff_ct,
m2_adr_ct,
m2_kast_ct,
m2_rating_ct,
m2_kills_t,
m2_deaths_t,
m2_kddiff_t,
m2_adr_t,
m2_kast_t,
m2_rating_t,
m3_kills_ct,
m3_deaths_ct,
m3_kddiff_ct,
m3_adr_ct,
m3_kast_ct,
m3_rating_ct,
m3_kills_t,
m3_deaths_t,
m3_kddiff_t,
m3_adr_t,
m3_kast_t,
m3_rating_t
from bronze.playerstable"""

In [ ]:
if not check_table_exist(silver_table):
    print("Creating Table")
    df = spark.sql(query.replace('Op,',''))
    df.write.mode("overwrite").saveAsTable(silver_table)
    print(f"{silver_table} criada")

else:
    print("Table already exists")

In [ ]:
def upsert_delta(df, batchId, query, delta_table, id_field, strongly_date):
    
    join = " and ".join([f'd.{i} = c.{i}' for i in id_field])
    
    w = window.Window.partitionBy(*id_field).orderBy(F.desc(strongly_date))
        
    df =(df.withColumn("Op", F.when(df._change_type == "insert","I")
                              .when(df._change_type == "update_preimage", "U")
                              .when(df._change_type == "update_postimage", "U")
                              .when(df._change_type == 'delete' ,"D"))
           .withColumn('rn', F.row_number().over(w))
           .filter('rn=1')
           .drop(F.col('rn')))
    
    view_name = f"{silver_table.split('.')[-1]}_view"
    df.createOrReplaceGlobalTempView(view_name)
    
    view_name = f"global_temp.{silver_table.split('.')[-1]}_view"
    cdc_data =  spark.sql(query.replace(bronze_table,view_name))
    
    (delta_table.alias("d")
                .merge(cdc_data.alias("c"), join) 
                .whenMatchedDelete(condition = "c.Op = 'D'")
                .whenMatchedUpdateAll(condition = "c.Op ='U'")
                .whenNotMatchedInsertAll(condition = "c.Op = 'I'")
                .execute())
    return None

In [ ]:
delta_table = DeltaTable.forName(spark, silver_table)

df_stream = (spark.readStream
                  .format("delta")
                  .option("readChangeFeed", "true")
                  .option("startingVersion", 0)
                  .table(bronze_table))

stream = (df_stream.writeStream
                   .format('delta')
                   .foreachBatch(lambda df, batchId: upsert_delta(df, batchId, query, delta_table, id_field, strongly_date))
                   .option('checkpointLocation', checkpoint_path)
                   .start())

In [ ]:
%sql select * from silver.playerstable where date = '2022-06-18'

date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,map_1,map_2,map_3,kills,assists,deaths,killad,hs,flash_assists,kast,kddiff,adr,fkdiff,rating,m1_kills,m1_assists,m1_deaths,m1_hs,m1_flash_assists,m1_kast,m1_kddiff,m1_adr,m1_fkdiff,m1_rating,m2_kills,m2_assists,m2_deaths,m2_hs,m2_flash_assists,m2_kast,m2_kddiff,m2_adr,m2_fkdiff,m2_rating,m3_kills,m3_assists,m3_deaths,m3_hs,m3_flash_assists,m3_kast,m3_kddiff,m3_adr,m3_fkdiff,m3_rating,kills_ct,deaths_ct,kddiff_ct,adr_ct,kast_ct,rating_ct,kills_t,deaths_t,kddiff_t,adr_t,kast_t,rating_t,m1_kills_ct,m1_deaths_ct,m1_kddiff_ct,m1_adr_ct,m1_kast_ct,m1_rating_ct,m1_kills_t,m1_deaths_t,m1_kddiff_t,m1_adr_t,m1_kast_t,m1_rating_t,m2_kills_ct,m2_deaths_ct,m2_kddiff_ct,m2_adr_ct,m2_kast_ct,m2_rating_ct,m2_kills_t,m2_deaths_t,m2_kddiff_t,m2_adr_t,m2_kast_t,m2_rating_t,m3_kills_ct,m3_deaths_ct,m3_kddiff_ct,m3_adr_ct,m3_kast_ct,m3_rating_ct,m3_kills_t,m3_deaths_t,m3_kddiff_t,m3_adr_t,m3_kast_t,m3_rating_t
2022-06-18T00:00:00.000+0000,refrezh,Streaming,FATE,Denmark,10610,2338077,5031,ESEA Advanced Season 32 Europe,3,Inferno,Dust2,null,43,10,27,26,14,4.0,76.8,16,84.2,3,1.32,14,7,9,7,4.0,90.0,5,81.0,0,1.31,29.0,7.0,18.0,7.0,4.0,69.4,11.0,85.9,3.0,1.34,null,null,null,null,null,null,null,null,null,null,27.0,7.0,20.0,120.1,91.3,2.0,16.0,20.0,-4.0,59.1,66.7,0.84,5.0,2.0,3.0,88.6,100.0,1.6,9.0,7.0,2.0,78.5,86.7,1.21,22.0,5.0,17.0,128.8,88.9,2.11,7.0,13.0,-6.0,43.0,50.0,0.56,null,null,null,null,null,null,null,null,null,null,null,null
2022-06-18T00:00:00.000+0000,ad3m,Streaming,fightclub,Sweden,13657,2325578,3918,MSI MGA 2018 Europe Closed Qualifier,1,Cache,null,null,19,4,15,8,5,2.0,68.2,4,81.4,2,1.15,19,4,15,5,2.0,68.2,4,81.4,2,1.15,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-06-18T00:00:00.000+0000,alter,Streaming,ex-Atmosphere,United States,14111,2323825,3688,ESEA MDL Season 28 North America,1,Dust2,null,null,26,2,21,7,19,null,67.9,5,102.5,0,1.24,26,2,21,19,null,67.9,5,102.5,0,1.24,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,13.0,10.0,3.0,109.5,73.3,1.24,13.0,11.0,2.0,94.5,61.5,1.25,13.0,10.0,3.0,109.5,73.3,1.24,13.0,11.0,2.0,94.5,61.5,1.25,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-06-18T00:00:00.000+0000,ToM223,Streaming,AGO,Poland,15300,2318047,3354,Legend Series #4,3,Overpass,Mirage,Train,64,18,60,22,32,2.0,73.9,4,76.5,-1,1.09,22,8,16,12,1.0,83.3,6,74.7,1,1.28,34.0,9.0,28.0,15.0,1.0,77.5,6.0,85.9,-3.0,1.18,8.0,1.0,16.0,5.0,0.0,50.0,-8.0,58.8,1.0,0.6,34.0,26.0,8.0,86.3,76.9,1.18,30.0,34.0,-4.0,68.8,71.4,1.01,15.0,7.0,8.0,86.3,86.7,1.41,7.0,9.0,-2.0,63.1,80.0,1.15,19.0,16.0,3.0,91.0,76.2,1.18,15.0,12.0,3.0,80.2,78.9,1.18,0.0,3.0,-3.0,52.3,33.3,0.22,8.0,13.0,-5.0,60.1,53.3,0.67
2022-06-18T00:00:00.000+0000,try,Evolve,Complexity,United States,19869,2323722,3721,ZOTAC Cup Masters 2018 North America Closed Qualifier,3,Inferno,Mirage,Nuke,44,6,49,1,19,1.0,63.5,-5,79.9,10,1.04,23,4,14,8,1.0,82.6,9,94.2,5,1.59,10.0,1.0,18.0,5.0,0.0,61.9,-8.0,60.1,2.0,0.73,11.0,1.0,17.0,6.0,0.0,42.1,-6.0,84.3,3.0,0.8,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2022-06-18T00:00:00.000+0000,NiKo,Streaming,E-frag.net,Bosnia and Herzegovina,3741,2300764,1958,Acer Predator Masters powered by Intel Season 2 Finals,3,Cache,Cobblestone,null,52,11,31,32,32,null,null,21,null,1,1.53,25,4,9,15,null,null,16,null,1,2.04,27.0,7.0,22.0,17.0,null,null,5.0,null,0.0,1.15,null,null,nu

In [ ]:
%sql SELECT * FROM table_changes('silver.playerstable', 0)

date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,map_1,map_2,map_3,kills,assists,deaths,killad,hs,flash_assists,kast,kddiff,adr,fkdiff,rating,m1_kills,m1_assists,m1_deaths,m1_hs,m1_flash_assists,m1_kast,m1_kddiff,m1_adr,m1_fkdiff,m1_rating,m2_kills,m2_assists,m2_deaths,m2_hs,m2_flash_assists,m2_kast,m2_kddiff,m2_adr,m2_fkdiff,m2_rating,m3_kills,m3_assists,m3_deaths,m3_hs,m3_flash_assists,m3_kast,m3_kddiff,m3_adr,m3_fkdiff,m3_rating,kills_ct,deaths_ct,kddiff_ct,adr_ct,kast_ct,rating_ct,kills_t,deaths_t,kddiff_t,adr_t,kast_t,rating_t,m1_kills_ct,m1_deaths_ct,m1_kddiff_ct,m1_adr_ct,m1_kast_ct,m1_rating_ct,m1_kills_t,m1_deaths_t,m1_kddiff_t,m1_adr_t,m1_kast_t,m1_rating_t,m2_kills_ct,m2_deaths_ct,m2_kddiff_ct,m2_adr_ct,m2_kast_ct,m2_rating_ct,m2_kills_t,m2_deaths_t,m2_kddiff_t,m2_adr_t,m2_kast_t,m2_rating_t,m3_kills_ct,m3_deaths_ct,m3_kddiff_ct,m3_adr_ct,m3_kast_ct,m3_rating_ct,m3_kills_t,m3_deaths_t,m3_kddiff_t,m3_adr_t,m3_kast_t,m3_rating_t,_change_type,_commit_version,_commit_timestamp
2022-06-18T00:00:00.000+0000,try,Evolve,Complexity,United States,19869,2323722,3721,ZOTAC Cup Masters 2018 North America Closed Qualifier,3,Inferno,Mirage,Nuke,44,6,49,1,19,1.0,63.5,-5,79.9,10,1.04,23,4,14,8,1.0,82.6,9,94.2,5,1.59,10.0,1.0,18.0,5.0,0.0,61.9,-8.0,60.1,2.0,0.73,11.0,1.0,17.0,6.0,0.0,42.1,-6.0,84.3,3.0,0.8,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,update_preimage,1,2022-06-21T05:31:12.000+0000
2022-06-18T00:00:00.000+0000,try,Evolve,Complexity,United States,19869,2323722,3721,ZOTAC Cup Masters 2018 North America Closed Qualifier,3,Inferno,Mirage,Nuke,44,6,49,1,19,1.0,63.5,-5,79.9,10,1.04,23,4,14,8,1.0,82.6,9,94.2,5,1.59,10.0,1.0,18.0,5.0,0.0,61.9,-8.0,60.1,2.0,0.73,11.0,1.0,17.0,6.0,0.0,42.1,-6.0,84.3,3.0,0.8,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,update_postimage,1,2022-06-21T05:31:12.000+0000
2018-03-29T00:00:00.000+0000,Friis,Memorable,Trust,Denmark,7,2321513,3644,Copenhagen Games 2018 BYOC Qualifier,3,Overpass,Inferno,Mirage,41,8,50,-1,14,2.0,66.2,-9,60.6,-1,0.85,6,4,16,2,2.0,55.6,-10,46.4,-1,0.46,18.0,3.0,18.0,4.0,0.0,69.0,0.0,67.0,-1.0,0.97,17.0,1.0,16.0,8.0,0.0,70.4,1.0,63.2,1.0,0.99,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,insert,0,2022-06-21T05:04:27.000+0000
2018-03-29T00:00:00.000+0000,Friis,Memorable,XI,Denmark,7,2321474,3644,Copenhagen Games 2018 BYOC Qualifier,3,Cache,Overpass,null,39,12,22,29,12,7.0,83.3,17,77.2,2,1.4,19,7,7,8,5.0,95.2,12,82.9,1,1.65,20.0,5.0,15.0,4.0,2.0,74.1,5.0,72.9,1.0,1.23,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,insert,0,2022-06-21T05:04:27.000+0000
2019-02-22T00:00:00.000+0000,RashiE,KoN Norway,KoN Sweden,Norway,13,2331189,4387,King of Nordic Season 13,1,Overpass,null,null,29,11,22,18,14,1.0,83.3,7,80.9,-1,1.35,29,11,22,14,1.0,83.3,7,80.9,-1,1.35,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,insert,0,2022-06-21T05:04:27.000+0000
2019-02-15T